In [1]:
from sympy import symbols, Matrix
from sympy import expand, factor #expand and factor expressions
import random
import numpy as np
from sympy import Matrix,eye,det,zeros,simplify
import time

[Sympy doc] (https://docs.sympy.org/latest/tutorials/intro-tutorial/index.html)

1. Symbolic computation: manipulating irrational numbers exactly using SymPy, computing symbolic expressions with variables.
2. most simplifications in Sympy are not performed automatically.
3. SymPy can simplify expressions, compute derivatives, integrals, and limits, solve equations, work with matrices, and much, much more, and do it all symbolically.



In [2]:
x,y=symbols('x y')
expr=x+2*y
expanded_expr=expand(x*expr)
factor(expanded_expr)

x*(x + 2*y)

some rules:
* To define variables, we must use symbols.
* if a variable is changed, expressions that were already created with that variable do not change automatically
* == represents exact structural equality testing. “Exact” here means that two expressions will compare equal with == only if they are exactly equal structurally.
* the best way to check whether a-b: subtract, simplify(b-a), and see whether you get 0 or a.equals(b)

In [2]:
x = symbols('x')
expr = x + 1
# x = 2
#print(expr)#changes made to x has no effects in expr, which is created before the chg
expr.subs(x, 2)

3

Converting Strings to SymPy Expressions
**sympify**, not to be confused with simplify

# Key Generation

In [3]:
def modify_coefficients(A,q):
    B=A.as_coefficients_dict()
    sparse_poly_mod=0
    for key, value in B.items():
        if value>q-1:
            if value%(q-1)==0:
                sparse_poly_mod+=(q-1)*key
            else:
                sparse_poly_mod+=(value%(q))*key
        else:
            sparse_poly_mod+=value*key
    return sparse_poly_mod


In [5]:
def poly_n_variables(n):
    K=[]
    for i in range(n):
        x='x'+str(i)
        K.append(x)
    return K

def sparse_polynomial(n,t,b,q):#t monomials, bound the degree of monomials by an integer b
    K=poly_n_variables(n)
    
    sparse_poly=0
    for i in range(t):
        deg=random.randint(1,b)
        monomoial_d=1
        
        for i in range(deg):
            monomoial_d*=symbols(random.choice(K))
            
        sparse_poly+=random.randint(1,q-1)*monomoial_d
        
#     print(sparse_poly)
    return modify_coefficients(sparse_poly,q)
     
#do some maniculation about the coefficient of the monomial, making sure it is less than= 5; if greater, modulo 6

sparse_polynomial(4,5,3,6).as_coefficients_dict()


defaultdict(int, {x3: 1, x3**3: 1, x2: 3, x1: 4, x2*x3**2: 5})

# Generate the matrix

In [9]:
def generate_U(k,n,t,B,q):
    I=eye(k)
    
    for i in range(int((k*k-k)/2)):
        b=random.randint(1,k-1)
        a=random.randint(0,b-1)
        I[a,b]=sparse_polynomial(n,t,B,q)
    return I
    
def generate_L(k,n,t,B,q):
    I=eye(k)
    
    for i in range(int((k*k-k)/2)):
        a=random.randint(1,k-1)
        b=random.randint(0,a-1)
        I[a,b]=sparse_polynomial(n,t,B,q)
    return I

A=generate_U(4,32,5,6,6)
B=A.inv(method="LU")
A*B

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

In [90]:
zeros(2,3)

Matrix([
[0, 0, 0],
[0, 0, 0]])

In [10]:

def col_permutation(matrix):
    col_permutation = random.sample(range(matrix.cols), matrix.cols)
#     permuted_matrix = matrix.permute_cols(col_permutation)
    
    I=zeros(len(col_permutation))
    for i in range(len(col_permutation)):
        I[i,col_permutation[i]]=1
#     print(I.inv())
#     print(matrix*I.inv())
    
    return I.inv()
col_permutation(Matrix([[1, 4, 7], [2, 5, 8], [3, 6, 9]]))


Matrix([
[1, 0, 0],
[0, 0, 1],
[0, 1, 0]])

In [13]:
def find_cols_rm(A,B):
    cols=[]
    for i in range(B.cols):
        for j in range(A.cols):
            if B[:,i]==A[:,j]:
                cols.append(j)
    col_rm=set(range(A.cols))
    for i in cols:
        col_rm.remove(i)
    return col_rm
            
find_cols_rm(Matrix([[5, 6, 6,8], [2, 2, 2,8], [6, 6, 2,8],[2,3,6,7]]),Matrix([
[5, 6, 6],
[2, 2, 2],
[6, 6, 2],
[2, 3, 6]]))

{3}

In [39]:

# def random_del_columns(k,l,matrix):
# #     col_indices = random.sample(range(matrix.cols), k-l)#ramdonly rm k-l cols
# #     print(col_indices)
#     temp=matrix.copy()
#     for i in range(int(k-l)):
#         j=random.randint(0,matrix.cols-1)
#         matrix.col_del(j)
#     col_rm=find_cols_rm(temp,matrix)
# #     print(col_rm)
# #     print(matrix)
#     return col_rm,matrix

# start = time.time()
# a,b=random_del_columns(4,3,Matrix([[5, 6, 6,8], [2, 2, 2,8], [6, 6, 2,8],[2,3,6,7]]))
# print(a)
# print(b)
Matrix([[5, 6, 6,8], [2, 2, 2,8], [6, 6, 2,8],[2,3,6,7]]).inv()

Matrix([
[-17,   -9,    12,  16],
[ 17, 35/4, -47/4, -16],
[ -4, -9/4,  11/4,   4],
[  1,  3/4,  -3/4,  -1]])

In [13]:
def random_del_columns(k,l,matrix):
    col_opts=[i for i in range(l)]
#     for i in range(k-l):
#         random_element=random.choice(col_opts)
#         col_opts.remove(random_element)
    
    matrix_kl=zeros(k,l)
    
    j=0
    for i in col_opts:
        matrix_kl[:,j]=matrix[:,i]
        j+=1
    return col_opts,matrix_kl

random_del_columns(4,2,Matrix([[5, 6, 6,8], [2, 2, 2,8], [6, 6, 2,8],[2,3,6,7]]))

([0, 1],
 Matrix([
 [5, 6],
 [2, 2],
 [6, 6],
 [2, 3]]))

In [10]:
def multiply_out(sample_M):
    for i in range(sample_M.rows):
        for j in range(sample_M.cols):
            sample_M[i,j]=expand(sample_M[i,j])
    return sample_M

multiply_out(Matrix([[1, 4], [2, 5, 8], [3, 6, 9]]))

Matrix([
[1, 4, 7],
[2, 5, 8],
[3, 6, 9]])

In [15]:
def find_kl_inverse(to_keep,S_inv):#find the inverse of kl_matrix
    
    k=S_inv.rows
    
#     to_keep=set(range(k))#rows to keep
    
#     for i in col_rm:
#         to_keep.remove(i)
        
    ml_inv=zeros(len(to_keep),k)
    
    j=0
    for i in to_keep:
        ml_inv[j,:]=S_inv[i,:]
        j+=1
    return ml_inv

find_kl_inverse([0,1],Matrix([[-17,-9,12,16],[17,35/4,-47/4,-16],[-4,-9/4,11/4,4],[1,3/4,-3/4,-1]]))

Matrix([
[-17,   -9,     12,  16],
[ 17, 8.75, -11.75, -16]])

In [24]:
def matrix_kl(k,l,s,n,t,B,q):#k,l are the dimensions of the matrix, s is the several, a small number
    Ls=[]
    Us=[]
    for i in range(s):
        U=generate_U(k,n,t,B,q)
        L=generate_L(k,n,t,B,q)
        Us.append(U)
        Ls.append(L)
        
    U_new=random.choice(Us)
    L_new=random.choice(Ls)
    Ms=Us+Ls
    
    P1=col_permutation(U_new)
    T1=random.choice(Ms)
    P2=col_permutation(U_new)
#     T2=random.choice(Ms)
    P3=col_permutation(U_new)
    
    
#     kk_matrix=U_new*L_new*P1*T1*P2*T2*P3
#     S_inv=P3.inv(method="LU")*T2.inv(method="LU")*P2.inv(method="LU")*T1.inv(method="LU")*P1.inv(method="LU")*L_new.inv(method="LU")*U_new.inv(method="LU")
#     kk_matrix=U_new*P1*L_new*P2*T1*P3
    kk_matrix=L_new*U_new
    kk=kk_matrix.copy()
    #S_inv=P3.inv(method="LU")*T1.inv(method="LU")*P2.inv(method="LU")*L_new.inv(method="LU")*P1.inv(method="LU")*U_new.inv(method="LU")
    S_inv=U_new.inv(method="LU")*L_new.inv(method="LU")
#     print(kk*S_inv)

    col_rm,kl_matrix=random_del_columns(k,l,kk_matrix)
    
    kl_inverse=find_kl_inverse(col_rm,S_inv)
    return kk,S_inv,kl_matrix, kl_inverse


start = time.time()
kk,A,B,C=matrix_kl(4,3,5,32,3,5,6)
end = time.time()
print(end - start)

0.17069506645202637


(3, 4)

In [25]:
substitution_ls=[]
K=poly_n_variables(32)
for i in range(0,32):
    substitution_ls.append((symbols(K[i]),random.randint(0,5)))

I_exp=B*C

In [26]:

for i in range(I_exp.rows):
    for j in range(I_exp.cols):
        I_exp[i,j]=I_exp[i,j].subs(substitution_ls)
        
I_exp

Matrix([
[   44438005,    -1571436,    18720,    -18],
[ 1733082156,   -61286003,   730080,   -702],
[39994203600, -1414292400, 16848001, -16200],
[    2468778,      -87302,     1040,      0]])

Matrix([
[1, 0, 0, 0, 0],
[0, 1, 0, 0, 0],
[0, 0, 1, 0, 0],
[0, 0, 0, 1, 0],
[0, 0, 0, 0, 1]])

In [43]:
x, y, z = symbols('x y z')
expand((x+1)*(x-1))

x**2 - 1

# Finding the inverse

In [28]:
M2=Matrix([[2,1],[7,4]])
M2.inv(method="LU")

# M2.adjugate()#the transpose of the matrix of cofactors.
# det(M2)

1

In [128]:
def vector_t_polynomials(n,l,t,b,q): #l is the number of polynomials, q the remainder of 256%l
    U=[]
    for i in range(l):
        P=sparse_polynomial(n,t,b,q)
        U.append(P)
    return U

print("generatingb l polynomials")
start = time.time()
matrix_l_poly=Matrix(vector_t_polynomials(32,5,4,5,6)).T
end = time.time()
print(end - start)

matrix_l_poly.shape


generatingb l polynomials
0.025541067123413086


(1, 5)

In [129]:
start = time.time()
D=matrix_l_poly*C
end = time.time()
print(end - start)

0.020200252532958984


Matrix([[0, 0, 0, 0, 0, 0, 0, x1*x12*x5**2 + 4*x11*x24*x27*x3*x7 + x13*x20*x27 + 5*x15, (-x0*x11 - 2*x0*x5**2 - 2*x15*x18*x28*x31 - 3*x26*x8)*(x1*x12*x5**2 + 4*x11*x24*x27*x3*x7 + x13*x20*x27 + 5*x15), (x1*x12*x5**2 + 4*x11*x24*x27*x3*x7 + x13*x20*x27 + 5*x15)*(-3*x0**2*x1*x25*x9 - 2*x12*x24*x26*x29 - x18 - x29)]])